In [1]:
#!pip install google-cloud-storage
import pandas as pd
from google.cloud import storage
# import os

# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "Hedwig/00_Data/msca310019-capstone-49b3-6876cdba6fac.json"

In [2]:
#!pip install pandarallel
import multiprocessing

num_processors = multiprocessing.cpu_count()
print(f'Available CPUs: {num_processors}')

import pandarallel
from pandarallel import pandarallel
pandarallel.initialize(nb_workers=num_processors-1, use_memory_fs=False)

Available CPUs: 8
INFO: Pandarallel will run on 7 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [2]:
client = storage.Client()
bucket = client.get_bucket('user-scripts-msca310019-capstone-49b3')

In [3]:
blob = bucket.blob('data/conversation_emails.csv')
content = blob.download_as_string()

In [3]:
import io
#df = pd.read_csv(io.BytesIO(content))
df = pd.read_csv('/Users/scottsmacbook/capstone/00_Data/data_conversation_emails.csv')

In [4]:
df.drop_duplicates(subset=['message'], keep='first',inplace=True)

In [5]:
thread_counts=df.groupby(['thread'], as_index=False)['message'].agg('count')
thread_counts.head(20)

,thread,message
0,0,8432
1,1,2546
2,2,68
3,3,137
4,4,1
5,5,2
6,6,2
7,7,3
8,8,3
9,9,4


In [6]:
thread_multi=thread_counts[thread_counts.message>=2]
thread_multi.drop([0,1], inplace = True)

/var/folders/p4/82nd8dg50fq_lzgh5s75s36h0000gn/T/ipykernel_56385/2027695490.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  thread_multi.drop([0,1], inplace = True)


In [7]:
multi_threads_list=thread_multi.thread.to_list()

In [8]:
df['flg_multi']=df['thread'].apply(lambda x: 1 if x in multi_threads_list else 0)

In [9]:
df_message_v3=df[df.flg_multi==1]
df_message_v3.head()

,Unnamed: 0,time,subject,thread,sender,recipient,cc,bcc,message,flg_multi
2,2,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",Re: test,2,0,[3],[],[],test successful. way to go!!!,1
4,4,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",Re: Hello,3,0,[5],[],[],Let's shoot for Tuesday at 11:45.,1
5,5,"Thu, 31 Aug 2000 04:17:00 -0700 (PDT)",Re: Hello,3,0,[5],[],[],"Greg, How about either next Tuesday or Thursda...",1
8,8,"Tue, 17 Oct 2000 02:26:00 -0700 (PDT)",Re: High Speed Internet Access,5,0,[9],[],[],1. login: pallen pw: ke9davis I don't think th...,1
9,9,"Mon, 16 Oct 2000 06:44:00 -0700 (PDT)",FW: fixed forward or other Collar floor gas pr...,6,0,[10],[],[],---------------------- Forwarded by Phillip K ...,1


In [10]:
df_message_v3['time'] = pd.to_datetime(df_message_v3['time'])
df_message_v3

/var/folders/p4/82nd8dg50fq_lzgh5s75s36h0000gn/T/ipykernel_56385/3744740549.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_message_v3['time'] = pd.to_datetime(df_message_v3['time'])


,Unnamed: 0,time,subject,thread,sender,recipient,cc,bcc,message,flg_multi
2,2,2000-10-18 03:00:00-07:00,Re: test,2,0,[3],[],[],test successful. way to go!!!,1
4,4,2000-08-31 05:07:00-07:00,Re: Hello,3,0,[5],[],[],Let's shoot for Tuesday at 11:45.,1
5,5,2000-08-31 04:17:00-07:00,Re: Hello,3,0,[5],[],[],"Greg, How about either next Tuesday or Thursda...",1
8,8,2000-10-17 02:26:00-07:00,Re: High Speed Internet Access,5,0,[9],[],[],1. login: pallen pw: ke9davis I don't think th...,1
9,9,2000-10-16 06:44:00-07:00,FW: fixed forward or other Collar floor gas pr...,6,0,[10],[],[],---------------------- Forwarded by Phillip K ...,1
...,...,...,...,...,...,...,...,...,...,...
245121,517393,2001-11-30 14:53:21-08:00,RE:,208,322,[51654],[],[],"not yet, rob has been in the court house all d...",1
245122,517394,2001-11-29 11:41:51-08:00,FW: Hello,11292,322,[51654],[],[],"-----Original Message----- From: \t""Lerner, L...",1
245123,517395,2001-11-29 11:04:24-08:00,RE:,208,322,[51636],[],[],"very busy, jess is ok just scared -----Origina...",1
245126,517398,2001-11-28 07:20:00-08:00,RE: CONFIDENTIAL,12455,322,[3541],[],[],"2 -----Original Message----- From: \tDoucet, D...",1


In [11]:
df_message_v3.sort_values(by=['thread','time'], ascending=[True,True], inplace=True)

/var/folders/p4/82nd8dg50fq_lzgh5s75s36h0000gn/T/ipykernel_56385/2072996508.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_message_v3.sort_values(by=['thread','time'], ascending=[True,True], inplace=True)


In [12]:
import ast
df_message_v3['recipient'] = df_message_v3['recipient'].apply(lambda x: ast.literal_eval(x))

/var/folders/p4/82nd8dg50fq_lzgh5s75s36h0000gn/T/ipykernel_56385/3840432164.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_message_v3['recipient'] = df_message_v3['recipient'].apply(lambda x: ast.literal_eval(x))


In [13]:
test = df_message_v3[df_message_v3.thread<31]

In [62]:
#test = test.explode('recipient')

In [14]:
df_message_v3.thread.unique()

array([     2,      3,      5, ..., 137844, 137870, 137877])

In [66]:
threads = df_message_v3.thread.unique()

message_replies = []
for thread in threads:
    thread_df = df_message_v3[df_message_v3.thread==thread]
    thread_df.drop(columns = ['Unnamed: 0','cc','bcc','flg_multi'],inplace=True)
    
    r = list(thread_df.recipient)
    s = list(thread_df.sender)
    
    pairs = []

    for i in range(len(thread_df)):
        for j in range(i,-1,-1):
            if(s[i] in r[j]):
                if(s[j] in r[i]):
                    pairs.append((j,i))
                    break

    for pair in pairs:
        message = thread_df.iloc[pair[0]]
        reply = thread_df.iloc[pair[1]]
        reply_new = reply.add_prefix('reply_')
        concatenated_row = pd.concat([message, reply_new], axis=0)
        message_replies.append(concatenated_row)
#     print(pairs)

In [58]:
message_replies_df = pd.DataFrame(data=message_replies)

In [62]:
message_replies_df.drop(columns = ['reply_subject','reply_thread'],inplace=True)

In [60]:
message_replies_df[message_replies_df.thread!=2]

,time,subject,thread,sender,recipient,message,reply_time,reply_subject,reply_thread,reply_sender,reply_recipient,reply_message
Unnamed 17,2000-10-06 00:42:00-07:00,RE: Hello,3,13140,[18237],the problem is that I need to go at noon.,2000-10-06 00:45:00-07:00,RE: Hello,3,18237,[13140],I'm more then happy to take you if you still n...
Unnamed 18,2000-10-06 00:42:00-07:00,RE: Hello,3,13140,[18237],the problem is that I need to go at noon.,2000-10-06 00:51:00-07:00,RE: Hello,3,18237,[13140],call me at work... 223-5600 x7804 -----Origina...
Unnamed 19,2001-05-01 09:42:00-07:00,RE: Hello,3,617,[4140],"Molly, Kim Watson would like us to bring Jacob...",2001-05-01 09:51:00-07:00,RE: Hello,3,4140,[617],Sounds great--I'll coordinate with Shirley. Ja...
Unnamed 20,2001-05-01 09:51:00-07:00,RE: Hello,3,4140,[617],Sounds great--I'll coordinate with Shirley. Ja...,2001-05-01 19:42:00-07:00,RE: Hello,3,617,[4140],"Molly, Kim Watson would like us to bring Jacob..."
Unnamed 21,2001-05-01 19:42:00-07:00,RE: Hello,3,617,[4140],"Molly, Kim Watson would like us to bring Jacob...",2001-05-01 19:51:00-07:00,RE: Hello,3,4140,[617],Sounds great--I'll coordinate with Shirley. Ja...
...,...,...,...,...,...,...,...,...,...,...,...,...
Unnamed 24661,2002-01-31 09:26:01-08:00,Canadian Trader List,137631,56,[322],"John, Please review the list below and let me ...",2002-01-31 09:34:39-08:00,RE: Canadian Trader List,137631,322,[56],Stephane Brodeur should be removed from the li...
Unnamed 24662,2002-01-31 09:34:39-08:00,RE: Canadian Trader List,137631,322,[56],Stephane Brodeur should be removed from the li...,2002-01-31 10:35:06-08:00,RE: Canadian Trader List,137631,56,[322],Thank you. Please let me know about Chris Lamb...
Unnamed 24663,2002-01-31 06:44:41-08:00,RE: Physical Gas,137634,322,[551],that was our plan but I am not sure whether UB...,2002-01-31 06:49:52-08:00,RE: Physical Gas,137634,551,[322],"John, I understand Peter Keohane is here today..."
Unnamed 24664,2002-01-30 09:40:23-08:00,Estate DPR Reporting for Power,137640,3595,"[3421, 322]",Peggy & John The estate's DPR reporting requir...,2002-01-30 10:09:47-08:00,RE: Estate DPR Reporting for Power,137640,322,[3595],"ok -----Original Message----- From: Reeves, Ka..."


In [64]:
message_replies_df.to_csv('/Users/scottsmacbook/capstone/00_Data/message_reply_pairs.csv',index=False)

In [65]:
message_replies_df[message_replies_df.thread==69]

,time,subject,thread,sender,recipient,message,reply_time,reply_sender,reply_recipient,reply_message
Unnamed 73,2000-10-25 03:42:00-07:00,Re: lunch,69,1340,[38500],"Hey, This week is probably not a good week, bu...",2000-10-25 04:51:00-07:00,38500,[1340],Great! I am best at scheduling things last min...
Unnamed 74,2000-11-27 03:04:00-08:00,RE: lunch,69,42586,[1691],Thurs.? -----Original Message----- From: Sara....,2000-11-27 05:15:00-08:00,1691,[42586],Thursday sounds great! I'll call you in the mo...
191581,2000-12-04 02:12:00-08:00,RE: lunch,69,18483,"[162, 18485, 18483]",Sorry to hear you're suffering. I'm flexible o...,2000-12-04 02:12:00-08:00,18483,"[162, 18485, 18483]",Sorry to hear you're suffering. I'm flexible o...
Unnamed 75,2001-10-05 09:01:41-07:00,RE: lunch,69,8558,[8600],Yes sure. I won't be here on Monday and possib...,2001-10-05 09:02:03-07:00,8600,[8558],sounds great. :) -----Original Message----- Fr...
Unnamed 76,2001-10-19 09:15:25-07:00,RE: lunch,69,1405,[24797],Shawn What about Tuesday next week? Vince ----...,2001-10-19 09:20:12-07:00,24797,[1405],"Vince, that sounds great. Should I drive by an..."
Unnamed 77,2001-10-19 09:20:12-07:00,RE: lunch,69,24797,[1405],"Vince, that sounds great. Should I drive by an...",2001-10-19 09:36:28-07:00,1405,[24797],"Shawn, Thanks. Tuesday next week, 12:00, it is..."
Unnamed 78,2001-11-19 14:36:01-08:00,RE: lunch,69,253,[43306],Sounds good. 11:30? -----Original Message-----...,2001-11-19 14:36:24-08:00,43306,[253],11:30 it is! -----Original Message----- From: ...
Unnamed 79,2001-11-20 06:24:30-08:00,RE: lunch,69,529,[20627],"I am going to Las Vegas for Thanksgiving, but ...",2001-11-20 06:40:13-08:00,20627,[529],I am open that week. How about Tuesday? From:\...
Unnamed 80,2001-11-20 06:40:13-08:00,RE: lunch,69,20627,[529],I am open that week. How about Tuesday? From:\...,2001-11-20 06:53:20-08:00,529,[20627],Good. Lets talk that morning. -----Original Me...
Unnamed 81,2001-10-19 09:20:12-07:00,RE: lunch,69,24797,[1405],"Vince, that sounds great. Should I drive by an...",2001-12-05 08:35:20-08:00,1405,[24797],"Shawn, I was about to call you. I have an emer..."


In [53]:
message_replies_df.thread.unique()[7]

69